In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import math
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

class Parameters_SEIR_mu_testing:
    
    def __init__(self, beta, N, t_E, t_I, mu, t_linspace):
    
        self.N = N
        self.sigma = 1 / t_E
        self.gamma = 1 / t_I
        self.mu = mu
        self.beta = beta
        self.t_linspace = t_linspace    

class SEIR_mu_testing:
    
    def __init__(self, param_obj):
        
        self.N = param_obj.N
        self.sigma = param_obj.sigma
        self.gamma = param_obj.gamma
        self.mu = param_obj.mu
        self.beta = param_obj.beta
        self.t_linspace = param_obj.t_linspace 
        self.sol = self.generate_solution(param_obj).y
        
    def fun(self, t, y, beta, sigma, gamma, mu):
        
        S, E, I, R = y
        
        d_S_dt = - beta * S * I
        d_E_dt = beta * S * I - (sigma + mu) * E
        d_I_dt = sigma * E - (gamma + mu) * I
        d_R_dt = gamma * I  + mu * (E + I)
        
        dydt = [d_S_dt, d_E_dt, d_I_dt, d_R_dt]
        
        return dydt
    
    def generate_solution(self, param_obj, y0 = None, method = 'LSODA'):
        
        # This generates a solution over a linear time space t_linspace
        
        if y0 == None:
            y0 = [param_obj.N - 1, 1, 0, 0]
        
        
        t0, tf = param_obj.t_linspace[0], param_obj.t_linspace[-1]
        
        return solve_ivp(self.fun, (t0, tf), y0, method, t_eval = param_obj.t_linspace, args = (param_obj.beta, param_obj.sigma, param_obj.gamma, param_obj.mu))

    def S(self):
        
        return self.sol[0]
    
    def E(self):
        
        return self.sol[1]
    
    def I(self):
        
        return self.sol[2]
    
    def R(self):
        
        return self.sol[3]
    
    def testing_reproductive_number(self, beta, N, sigma, gamma, mu):
        
        return sigma * gamma / ( (sigma + mu) * (gamma + mu) )
        

In [2]:
R0 = 3
N = 8000000
t_E, t_I = 1, 3
mu = 0.01
t_linspace = np.linspace(0, 80, 100 * 100)

beta = R0 / (N * t_I)

params_0 = Parameters_SEIR_mu_testing(beta, N, t_E, t_I, mu, t_linspace)
SEIR_0 = SEIR_mu_testing(params_0)

In [11]:
# Interactive plot of the testing reproductive number vs mu

# Plot interactive graph of the proportions of the infected population that are E and I

R0 = 3

def interact_plot_testing_reproductive_number(mu, T):
    
    N = 8000000
    alpha_linspace = np.linspace(0.01, 0.99, 1000)
    beta = R0 / (N * t_I)
    
    
    mu_linspace = np.linspace(0, 0.5, 1000)
    
    testing_R0s = [R0 * SEIR_0.testing_reproductive_number(R0 / (N * (1 - alpha)* T), N, 1 / (alpha * T), 1 / ((1 - alpha) * T), mu) for alpha in alpha_linspace]
    
    fig, ax = plt.subplots()
    ax.plot(mu_linspace, testing_R0s)
    ax.set_ylim((0, 4))
    ax.set_xlabel(r"$\alpha$")
    ax.set_title(r'Plot of $\hat{\mathbb{R}_0}$')
    plt.show()
    
interact(interact_plot_testing_reproductive_number, mu = (0, 0.3, 0.01), T = (1, 20, 1))
    

interactive(children=(FloatSlider(value=0.15, description='mu', max=0.3, step=0.01), IntSlider(value=10, descr…

<function __main__.interact_plot_testing_reproductive_number(mu, T)>

In [4]:
# Interactive plot of the testing reproductive number vs mu

# Plot interactive graph of the proportions of the infected population that are E and I

R0 = 3

def interact_plot_testing_reproductive_number(alpha, T):
    
    N = 8000000
    t_linspace = np.linspace(0, 200, 100 * 100)
    t_E = alpha * T
    t_I = (1 - alpha) * T
    beta = R0 / (N * t_I)
    
    
    mu_linspace = np.linspace(0, 0.5, 1000)
    
    testing_R0s = [R0 * SEIR_0.testing_reproductive_number(beta, N, 1 / t_E, 1 / t_I, mu) for mu in mu_linspace]
    
    fig, ax = plt.subplots()
    ax.plot(mu_linspace, testing_R0s)
    ax.set_ylim((0, 3))
    plt.show()
    
interact(interact_plot_testing_reproductive_number, alpha = (0.01, 0.99, 0.01), T = (1, 20, 1))
    

interactive(children=(FloatSlider(value=0.5, description='alpha', max=0.99, min=0.01, step=0.01), IntSlider(va…

<function __main__.interact_plot_testing_reproductive_number(alpha, T)>